In [15]:
import torch
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, Sequential, Module
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

In [16]:
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        # self.se1 = SEBlock(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)        

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(1024, 2048)
        self.fc2 = nn.Linear(2048, 1024) 
        self.dropout1 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1024, 6)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        # x = self.se1(x)
        # x = F.relu(self.se1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.max_pool2d(x, 2)
        
        x = self.pool(x)
        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [22]:
# import training model
model = EmotionClassifier()

model.load_state_dict(torch.load('best_vgg.pth'))
# model.eval()


<All keys matched successfully>

In [29]:
# Define the transformation pipeline
class_labels = ['happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear']

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_and_transform_image(image_path):
    try:
        image = Image.open(image_path)
        transformed_image = transform(image)
        # Add a batch dimension
        transformed_image = transformed_image.unsqueeze(0)
        return transformed_image
    except Exception as e:
        print("Error loading or transforming the image:", e)
        return None
    
test_path = '/Users/kawka/Documents/GitHub/SEP-CVDL/dataset/trainingsets_64x64/RAF-DB_15.000_test/happiness/test_0003_aligned.jpg'

# Load and transform the image
transformed_image = load_and_transform_image(test_path)

if transformed_image is not None:
    print("Image loaded and transformed successfully!")
    # Optionally, you can further process or display the transformed image here
else:
    print("Failed to load or transform the image.")

output = model(transformed_image)
print(output)
probabilities = F.softmax(output, dim=1)
print(probabilities)
rounded_probabilities = torch.round(probabilities * 100) / 100
print(rounded_probabilities)

Image loaded and transformed successfully!
tensor([[-189.1484,  -22.7918,  -16.7523,  -20.3876,   -8.6374,  -15.7019]],
       grad_fn=<AddmmBackward0>)
tensor([[0.0000e+00, 7.1176e-07, 2.9870e-04, 7.8787e-06, 9.9884e-01, 8.5391e-04]],
       grad_fn=<SoftmaxBackward0>)
tensor([[0., 0., 0., 0., 1., 0.]], grad_fn=<DivBackward0>)
